# Your name: 

This notebook implements timeloop/accelergy-based energy estimation for the neural network model you trained. This part has to be run with the docker we provide, and does not require GPU support. 

One strategy to reduce the profiling time is to design a model with repeated layers since layers with the same architecture only need one time of profiling.
The profiler will also automatically save the information of profiled layers to a .json file specifiled by `profiled_lib_dir`, so that next time the same layer is profiled, the results can be obtained immediately. 


## Load the model

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [10]:
# define your own model [TODO]
# HINT : You might want to consider a network using depth-wise convolution and residual connection!

class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride):
        super(InvertedResidual, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
            nn.BatchNorm2d(oup),
            nn.ReLU6(inplace=True),
            nn.Conv2d(oup, oup, 3, 1, 1, bias=False),
            nn.BatchNorm2d(oup),
        )
        self.downsample = nn.Sequential(
            nn.Conv2d(inp, oup, 1, stride=stride, bias=False),
            nn.BatchNorm2d(oup),
        )

    def forward(self, x):
        residual = x
        x = self.conv(x)
        x += self.downsample(residual)
        x = F.relu6(x)
        return x


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # build a network with InvertedResidual
        layers = []
        layers.append(nn.Conv2d(3, 16, 3, 1, 1, bias=False))
        layers.append(nn.BatchNorm2d(16))
        layers.append(nn.ReLU6(inplace=True))
        layers.append(InvertedResidual(16, 32, 2))
        layers.append(InvertedResidual(32, 64, 2))
        layers.append(InvertedResidual(64, 64, 2))
        layers.append(InvertedResidual(64, 128, 2))
        layers.append(nn.AdaptiveAvgPool2d((1, 1)))
        layers.append(nn.Flatten())
        layers.append(nn.Linear(128, 10))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [11]:
model = Net()

## Run the Profiler to estimate the peak activation size

In [12]:
from profiler import count_activation_size
peak_activation_size = count_activation_size(
    net=model,
    input_size=(1, 3, 32, 32),
)

print("Peak Activation Sizes: {} Byte".format(peak_activation_size))

Peak Activation Sizes: 98304.0 Byte


OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

## Run the Profiler for Timeloop/Accelergy

In [13]:
from profiler import Profiler
from datetime import date

today = date.today()
sub_dir = "network-" + today.strftime("%b-%d-%Y")

profiler = Profiler(
    top_dir='workloads',
    sub_dir=sub_dir,
    timeloop_dir='simple_weight_stationary',
    model=model,
    input_size=(3, 32, 32),
    batch_size=1,
    convert_fc=True,
    exception_module_names=[],
    profiled_lib_dir=f"profiled_lib.json"
)

layer_wise_info, overall = profiler.profile()

for layer_id, info in layer_wise_info.items():
    print("Name: {} \t Energy: {:.2f} \t Cycle: {} \t Number of same architecture layers: {}".format(
        info['name'], info['energy'], info['cycle'], info['num']))

print()
print("Total Energy: {:.8f} mj".format(overall['total_energy'] / 1e3))
print("Total Cycles: {:.8f} Million".format(overall['total_cycle'] / 1e6))
print("MACs: {}".format(overall['macs']))
print("Num of Parameters: {}".format(overall['num_params']))
print("Peak Activation Size: {} Byte".format(overall['activation_size']))

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

converting nn.Conv2d and nn.Linear in network-Feb-28-2023 model ...
unknown module type <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
unknown module type <class 'torch.nn.modules.activation.ReLU6'>
unknown module type <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
unknown module type <class 'torch.nn.modules.activation.ReLU6'>
unknown module type <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
unknown module type <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
unknown module type <class '__main__.InvertedResidual'>
unknown module type <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
unknown module type <class 'torch.nn.modules.activation.ReLU6'>
unknown module type <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
unknown module type <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
unknown module type <class '__main__.InvertedResidual'>
unknown module type <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
unknown module type <class 'torch.nn.modules.activation.ReLU6'>
unknown modu

100%|██████████| 11/11 [02:32<00:00, 13.82s/it]

timeloop running finished!
Name: /home/workspace/lab1/4_training_network/workloads/network-Feb-28-2023/network-Feb-28-2023_layer1 	 Energy: 40.26 	 Cycle: 9216 	 Number of same architecture layers: 1
Name: /home/workspace/lab1/4_training_network/workloads/network-Feb-28-2023/network-Feb-28-2023_layer2 	 Energy: 31.90 	 Cycle: 4608 	 Number of same architecture layers: 1
Name: /home/workspace/lab1/4_training_network/workloads/network-Feb-28-2023/network-Feb-28-2023_layer3 	 Energy: 61.11 	 Cycle: 9216 	 Number of same architecture layers: 1
Name: /home/workspace/lab1/4_training_network/workloads/network-Feb-28-2023/network-Feb-28-2023_layer4 	 Energy: 3.11 	 Cycle: 512 	 Number of same architecture layers: 1
Name: /home/workspace/lab1/4_training_network/workloads/network-Feb-28-2023/network-Feb-28-2023_layer5 	 Energy: 33.65 	 Cycle: 4608 	 Number of same architecture layers: 1
Name: /home/workspace/lab1/4_training_network/workloads/network-Feb-28-2023/network-Feb-28-2023_layer6 	 Energ


OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Dete